In [ ]:
import os, sys 
sys.path.append(r"C:\Users\YangF\Desktop\PROJECT")

import pImage
import math
import matplotlib.pyplot as plt
import numpy as np
from nptdms import TdmsFile

In [ ]:
date_manip = '20221221'
animal_list = 'FY8_2'
hiris_name = ''
# read folder
hiris_folder_base = r'\\157.136.60.15\EqShulz\Fan\DATA_MICE_FAN\Batch_DECEMBER2022\hiris_raw'
labview_folder_base = r'\\157.136.60.15\EqShulz\Fan\DATA_MICE_FAN\Batch_DECEMBER2022\LABVIEW'
hiris_file_folder = os.path.join(hiris_folder_base,date_manip,animal_name,hiris_name)
labview_dir = os.path.join(labview_folder_base,date_manip,animal_name)

In [ ]:
input_labview_path = os.path.joinlabview_dir,'AI.tdms')
input_video_path = os.path.join(hiris_file_folder,'sequence.seq')
output_video_folder = os.path.join(r'\\157.136.60.15\EqShulz\Fan\DATA_MICE_FAN\Batch_DECEMBER2022\HIRIS',date_manip,animal_name,'movie')
print(input_labview_path)
print(input_video_path)
print(output_video_folder)

In [ ]:
def edge_detection(signal,threshold):
    rising_edge_index = np.where((signal [:-1] < threshold) & (signal [1:] > threshold))
    rising_edge_index = rising_edge_index[0]+1
    falling_edge_index = np.where((signal [:-1] > threshold) & (signal [1:] < threshold))
    falling_edge_index = falling_edge_index[0]
    return rising_edge_index,falling_edge_index

In [ ]:
AI_data = TdmsFile.read(input_labview_path)
group_name = AI_data.groups()
group_channels = AI_data['Untitled'].channels()
imacq = (AI_data['Untitled']['Untitled 13'][:]>trigger_val ).astype(int)
imacq = desample_signal(imacq,bin_size)

In [ ]:
# read task parameters to read the labview analog sampling rate
f = open(os.path.join(labview_dir,"trainingPara"),'r')
Task_data = f.readlines()
for para in Task_data:
    TASK_data = para.split(' ')
fs_labview = int(float(TASK_data[12]))*2
image_duration = 4096 #int(float(TASK_data[15])*500/1000)
ntrial = int(float(TASK_data[17]))

In [ ]:
imacq= np.array(imacq)
imacq_start,_ = edge_detection(imacq,0.5)
# check if the amount of recorded trials 
if np.shape(imacq_start)[0] != np.shape(micam_seq)[0]:
    print('ERROR')
imacq_end = imacq_start + image_duration

In [ ]:
# read trial finish time stamps from Labview
tdms_file = TdmsFile.read(input_labview_path)
channel = tdms_file['Untitled']['Untitled 2']
trial_finish = channel[:]/fs_clock
# trial finsih time in second
trial_finish_sec = trial_finish[trial_finish != 0]
ntrial = len(trial_finish_sec)
print(ntrial)
# trial finish with frame number
trial_finish_frame = np.floor(trial_finish_sec*fs_hiris)
trial_finish_frame = trial_finish_frame.astype(int)
trial_start_frame = np.insert(trial_finish_frame[:-1]+1,0,0)